In [14]:

import pandas as pd
from google.cloud import speech
import os

def transcribe_audio(file_path) -> object:
	client = speech.SpeechClient()

	with open(file_path, "rb") as audio_file:
		content = audio_file.read()

	audio = speech.RecognitionAudio(content=content)
	config = speech.RecognitionConfig(
		language_code="en-US", 
	)

	response = client.recognize(config=config, audio=audio)

	for result in response.results:
		print(f"Transcript: {result.alternatives[0].transcript}")

	return response

current_path = os.getcwd()
transcribe_audio(os.path.join(current_path, 'sample_customer_call.wav'))

Transcript: hello I'm experiencing an issue with your product I'd like to speak to someone about a replacement


results {
  alternatives {
    transcript: "hello I\'m experiencing an issue with your product I\'d like to speak to someone about a replacement"
    confidence: 0.997099459
  }
  result_end_time {
    seconds: 7
    nanos: 20000000
  }
  language_code: "en-us"
}
total_billed_time {
  seconds: 8
}
request_id: 316240562564634978

In [ ]:
transcript1 = result

df = pd.read_csv('customer_call_transcriptions.csv')
df

,text,sentiment_label
0,how's it going Arthur I just placed an order w...,negative
1,yeah hello I'm just wondering if I can speak t...,neutral
2,hey I receive my order but it's the wrong size...,negative
3,hi David I just placed an order online and I w...,neutral
4,hey I bought something from your website the o...,negative
...,...,...
97,yeah hello I'm just wondering if I can speak t...,neutral
98,hi I recently ordered a new phone and I'm just...,neutral
99,just looking to get some more information on t...,neutral
100,hi I just realised I ordered the wrong compute...,negative
